In [12]:
import json 
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from gurobipy import quicksum
import plotly.graph_objects as go

with open('generated_instances.json') as f:
  loaded_instances = json.load(f)

# instance is an object containing generated instance parameters
def solveModel(instance, iteration):
    # STEP 1: DEFINE THE MODEL
    m = gp.Model()
    
    # STEP 2: DEFINE THE PARAMETERS THAT WILL BE USED
    
    #number of tables
    N = instance['num_tables']
    print("Number of tables:", N)
    
    #Minimum physical distance required between tables
    d = 2
    
    #capacity for each table
    c = [table['capacity'] for table in instance['tables']]
    
    #width for each table
    w = [table['width'] for table in instance['tables']]
    print("Table widths:", w)
   
    #account for physical distancing margin
    for i in range(len(w)):
        w[i] += d

    #height for each table
    h = [table['height'] for table in instance['tables']]
    print("Table heights:", h)
    
    #account for physical distancing margin
    for i in range(len(h)):
        h[i] += d
    
    #Width of dining space
    Rw = instance['width']

    #Height of dining space
    Rh = instance['height']
    
    #Unused table space goal
    U = 2

    #Minimum desired establishment capacity goal
    C = 

    #Underperformance measure of staff when table i and j are used Uij[rows][columns]
    Uij =  [[1 for table in instance['tables']] for table in instance['tables'] ]
    print("Table underperformance matrix:", Uij)
  
    # STEP 3: DEFINE DECISION VARIABLES
    
    #binary, if table i is used
    f = {}
    for i in range(N):
        f[i] = m.addVar(vtype=GRB.BINARY, name="f_" +str(i))

    #bottom left x coordinate of table i
    x = {}
    for i in range(N):
        x[i] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i))

    #bottom left y coordinate of table i
    y = {}
    for i in range(N):
        y[i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_"+str(i))


    #binary, if table i is located left to table j
    l = {}
    for i in range(N):
        for j in range(N):
            if i != j:
                l[i, j] = m.addVar(vtype=GRB.BINARY, name="l_" +str(i)+str(j))

    #binary, if table i is located below to table j
    b = {}
    for i in range(N):
        for j in range(N):
            if i != j:
                b[i, j] = m.addVar(vtype=GRB.BINARY, name="b_" +str(i)+str(j))

    #binary, if table i and table j are both used
    g = {}
    for i in range(N):
        for j in range(N):
            if i != j:
                g[i, j] = m.addVar(vtype=GRB.BINARY, name="g_" +str(i)+str(j))

    #surplus for goal 1  
    g1={}
    g1 = m.addVar(vtype=GRB.CONTINUOUS, name="g1")

    #slack/surplus for goal 2
    g2={}
    g2= m.addVar(vtype=GRB.CONTINUOUS, name="g2")
    
    
    # STEP 4: DEFINE OBJECTIVE FUNCTION
    nonlinearobj = quicksum((g[i, j]*Uij[i][j])/2 for i in range(N) for j in range(N) if i != j)
    m.setObjective(g1+g2+(1/2*nonlinearobj), GRB.MINIMIZE)
    
    # Step 5: ADD CONSTRAINTS
    # no overlap
    m.addConstrs(l[i,j] + l[j,i] + b[i,j] + b[j,i] + (1-f[i]) + (1-f[j])  >= 1 for i in range(N) for j in range(N) if i != j)

    # If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate
    m.addConstrs(x[i] + w[i] <= (x[j] + Rw - Rw*l[i,j]) for i in range(N) for j in range(N) if i != j)

    #If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate
    m.addConstrs(y[i] + h[i] <= (y[j] + Rh - Rh*b[i,j]) for i in range(N) for j in range(N) if i != j)

    #table space must exist within the dining space
    m.addConstrs(x[i] >= 0 for i in range(N))
    m.addConstrs(x[i]+w[i]<= Rw for i in range (N))
    m.addConstrs(y[i] >= 0 for i in range(N))
    m.addConstrs(y[i]+h[i]<= Rh for i in range (N))

    #Unused table space must be under U - GOAL 1
    m.addConstrs((Rw*Rh) - quicksum((w[i])*(h[i])*f[i] for i in range(N)) - g1 == U for i in range(N))

    #Facility capacity must be over C to remain profitable - GOAL 2
    m.addConstrs(quicksum(f[i]*c[i] for i in range(N)) + g2 == C for i in range(N))

    #Quadratic constraint
    m.addConstrs(g[i,j] >= f[i] + f[j] - 1 for i in range(N) for j in range(N) if i != j)

    #update model
    m.update()
    
    # STEP 6: SOLVE THE MODEL
    m.optimize()
    
    # STEP 7: VISUALLY DISPLAY RESULTS
    #Parameters
    print("Room size:", Rw*Rh, "m^2")
    #Tables:
    for i in f:
        print("Table", i, "was", "not" if i==0 else "", "used.", "Position:", "(", x[i].x,",",y[i].x,").", "Capacity:", c[i], "Table space height:", h[i], "Table space width:", w[i])
    #Goal 1 print
    print("Goal 1: Unused space must be under", U, "m^2.", )
    print("Goal 1 Result: Unused space was", g1.x, "m^2.", )
    
    #Goal 2 print
    print("Goal 2: Facility capacity must be over", C, "to remain profitable.", )
    print("Goal 2 Result: Facility capacity was unmet by", g2.x, ".", )
    
    #nonlinear element print
    print("Underperformance level for this table setup is", nonlinearobj.getValue())
    tableUsedData = []
    
#     for tableUsed in f:
#         tableUsedData.append(f[tableUsed].x)
#     xCoords = []
#     for i in x:
#         xCoords.append(x[i].x)
#     yCoords = []
#     for i in y:
#         yCoords.append(y[i].x)
#     d = {'ifUsed' : tableUsedData, 'x-coord' : xCoords, 'y-coord' : yCoords, 'capacity' : c, 'table space width' : w, 'table space height' : h}
#     df = pd.DataFrame(data=d)
    
    # STEP 8: CREATE VISUALIZATION USING PLOTLY
    xCoordPlot = []
    yCoordPlot = []
    for i in f:
        if(f[i].x == 1):
            #top border
            xCoordPlot.append(0)
            xCoordPlot.append(Rw)
            yCoordPlot.append(Rh)
            yCoordPlot.append(Rh)
            xCoordPlot.append("none")
            yCoordPlot.append("none")

            #right border
            xCoordPlot.append(Rw)
            xCoordPlot.append(Rw)
            yCoordPlot.append(0)
            yCoordPlot.append(Rh)
            xCoordPlot.append("none")
            yCoordPlot.append("none")
            #x
            xCoordPlot.append(x[i].x)
            xCoordPlot.append(x[i].x)
            xCoordPlot.append(x[i].x + w[i])
            xCoordPlot.append(x[i].x + w[i])
            xCoordPlot.append(x[i].x)
            xCoordPlot.append("none")
            #y
            yCoordPlot.append(y[i].x)
            yCoordPlot.append(y[i].x + h[i])
            yCoordPlot.append(y[i].x + h[i])
            yCoordPlot.append(y[i].x)
            yCoordPlot.append(y[i].x)
            yCoordPlot.append("none")
            
    fig = go.Figure(go.Scatter(x=xCoordPlot, y=yCoordPlot, fill="toself"))
    fig.show()
    # STEP 9: SAVE MODEL, SOLUTION, PLOTLY FIGURE, AND DECISION VARIABLE VALUES TO /solved_models/ DIRECTORY
   # m.write('solved_models/instance_%d_out.lp' % iteration)
    #m.write('solved_models/instance_%d_out.json' % iteration)
    #fig.write_image('solved_models/instance_%d_fig.png' % iteration, engine='kaleido')
    #df.to_csv('solved_models/instance_%d_out.csv' % iteration, index=False)

#testInstance = {'restaurant_size': 'small', 'num_tables': 3, 'width': 7, 'height': 7, 'tables': [{'size': 'small', 'width': 1, 'height': 1, 'capacity': 2}, {'size': 'medium', 'width': 1, 'height': 2, 'capacity': 2}, {'size': 'large', 'width': 2, 'height': 2, 'capacity': 2}]}
testInstance = {"restaurant_size": "small", "num_tables": 5, "width": 15, "height": 13, "tables": [{"size": "large", "width": 4, "height": 3, "capacity": 3}, {"size": "medium", "width": 3, "height": 3, "capacity": 3}, {"size": "large", "width": 3, "height": 4, "capacity": 3}, {"size": "small", "width": 2, "height": 2, "capacity": 3}, {"size": "small", "width": 2, "height": 2, "capacity": 3}]}
solveModel(testInstance, 1)
# iteration = 0
# for instance in loaded_instances:
#     print(instance)
#     solveModel(instance, iteration)
#     iteration+=1

Number of tables: 5
Table widths: [4, 3, 3, 2, 2]
Table heights: [3, 3, 4, 2, 2]
Table underperformance matrix: [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 110 rows, 77 columns and 380 nonzeros
Model fingerprint: 0xd358a945
Variable types: 12 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 203.0000000
Presolve removed 49 rows and 12 columns
Presolve time: 0.00s
Presolved: 61 rows, 65 columns, 215 nonzeros
Variable types: 10 continuous, 55 integer (55 binary)

Root relaxation: objective 1.105000e+02, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 